In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




In [ ]:
# прочитаем из csv-файла данные с описанием автомобилей и их потреблением, распечатаем его размер и первые 5 строк
cars = pd.read_csv('/content/drive/MyDrive/DATASETS/auto_cons.csv')


# уберём строки с пустыми значениями из набора данных
cars.dropna(inplace = True)

# сохраним датафрейм с учётом преобразования признаков
cars = pd.get_dummies(cars)

# разделим наши данные на признаки (матрица X) и целевую переменную (y)
X = cars.drop(columns = ['Расход топлива'])
y = cars['Расход топлива']


In [ ]:

# разделяем модель на обучающую и валидационную выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# создадим объект класса StandardScaler и применим его к обучающей выборке
scaler = StandardScaler()
X_train_st = scaler.fit_transform(X_train) #обучаем scaler и одновременно трансформируем матрицу для обучающей выборки

# применяем стандартизацию к матрице признаков для тестовой выборки
X_test_st = scaler.transform(X_test)



In [ ]:
# задайте список моделей
models = [Lasso(), Ridge(), DecisionTreeRegressor(), RandomForestRegressor(), GradientBoostingRegressor()]

# функция, которая вычисляет MAPE
def mape(y_true, y_pred):
    y_error = y_true - y_pred
    y_error_abs = [abs(i) for i in y_error]
    perc_error_abs = y_error_abs / y_true
    mape = (perc_error_abs.sum() / len(y_true))
    return mape

# функция, которая принимает на вход модель и данные и выводит метрики
def make_prediction(m, X_train, y_train, X_test, y_test):
    model = m
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('MAE:{:.2f} MSE:{:.2f} MAPE:{:.2f} R2:{:.2f} '.format(mean_absolute_error(y_test, y_pred), 
                                          mean_squared_error(y_test, y_pred),
                                                                    mape(y_test, y_pred),
                                                                    r2_score(y_test, y_pred)))
# напишите цикл, который выводит метрики по списку моделей
for i in models:
    print(i)
    make_prediction(i, X_train_st, y_train, X_test_st, y_test)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
MAE:1.37 MSE:3.18 MAPE:0.15 R2:0.78 
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
MAE:0.94 MSE:1.75 MAPE:0.09 R2:0.88 
DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')
MAE:0.91 MSE:2.20 MAPE:0.09 R2:0.85 
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
        

In [ ]:
# обучим финальную модель
final_model = GradientBoostingRegressor()
final_model.fit(X_train_st, y_train)

y_pred = final_model.predict(X_test_st)
#print(final_model.feature_importances_)     
# создадим датафрейм с именами признаков и их важностью и выведем его по убыванию важности
df = pd.DataFrame({'feature': X_train.columns, 'coeff' : final_model.feature_importances_})
print(df.sort_values(by='coeff', ascending=False))

                   feature     coeff
1  Рабочий объем двигателя  0.422260
3                      Вес  0.265446
2                 Мощность  0.165734
5              Год выпуска  0.085182
0         Кол-во цилиндров  0.037503
4                Ускорение  0.020654
8         Производитель_US  0.002795
7     Производитель_Europe  0.000298
6       Производитель_Asia  0.000128
